<a href="https://colab.research.google.com/github/JackyYang27/data-science-and-big-data/blob/main/compare_between_required_and_elective_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# 學期代碼列表
semesters = [
    "1021(01)", "1022(01)", "1031(01)", "1032(01)", "1041(01)", "1042(01)",
    "1051(01)", "1052(01)", "1061(01)", "1062(01)", "1071(01)", "1072(01)"
]

# 部門列表
departments = ['資訊管理學系', '文學院學士班', '中國文學系', '英美語文學系', '法國語文學系',
    '理學院學士班', '物理學系', '數學系', '化學學系', '光電科學與工程學系',
    '工學院學士班', '化學工程與材料工程學系', '土木工程學系', '機械工程學系',
    '企業管理學系', '資訊管理學系', '財務金融學系', '經濟學系',
    '資訊電機學院學士班', '電機工程學系', '資訊工程學系', '通訊工程學系',
    '地球科學學院學士班', '地球科學學系', '大氣科學學系', '太空科學與工程學系',
    '客家語文暨社會科學學系', '生命科學系', '生醫科學與工程學系']

# 定義加權平均函數
def weighted_average(df, course_type):
    relevant_courses = df[(df['課程屬性名稱'] == course_type)]
    if not relevant_courses.empty:
        return (relevant_courses['成績'] * relevant_courses['學分數']).sum() / relevant_courses['學分數'].sum()
    else:
        return np.nan

# 遍歷每個學期進行分析
for semester in semesters:
    # 讀取學生平均成績數據
    url_1 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E5%AD%B8%E6%9C%9F%E5%B9%B3%E5%9D%87%E6%88%90%E7%B8%BE_{semester}.csv"
    df_1 = pd.read_csv(url_1)
    df_filtered_1 = df_1[df_1['系所名稱'].isin(departments)]
    df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')
    df_filtered_1 = df_filtered_1.dropna(subset=['學期-班一般人數', '學期-班一般名次'])

    # 初始化數據框架
    top_students = pd.DataFrame()

    # 分組並識別前5%的學生
    for (dept, year_class), group in df_filtered_1.groupby(['系所名稱', '年級班別']):
        percentile_5 = max(int(round(len(group) * 0.05)), 1)
        top_students_in_group = group.nsmallest(percentile_5, '學期-班一般名次')

        top_students = pd.concat([top_students, top_students_in_group], ignore_index=True)

    # 獲取前5%學生的學號列表
    top_student_ids = top_students['學號'].tolist()

    # 讀取選課資料
    url_2 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E9%81%B8%E8%AA%B2%E6%88%90%E7%B8%BE%E8%B3%87%E6%96%99_{semester}.csv"
    df_2 = pd.read_csv(url_2)

    # 篩選出前5%學生的選課資料
    selected_courses_top = df_2[df_2['學號'].isin(top_student_ids)]

    # 計算必修科目的加權平均成績
    required_course_average = weighted_average(selected_courses_top, '必修')

    # 計算選修和通識科目的加權平均成績
    elective_course_average = weighted_average(selected_courses_top, '選修')
    general_course_average = weighted_average(selected_courses_top, '通識')

    elective_and_general_average = np.nanmean([elective_course_average, general_course_average])

    # 輸出結果
    print(f"{semester} 學期 - 前5%學生平均成績：")
    print(f"必修科目加權平均成績: {required_course_average:.2f}")
    print(f"選修和通識科目加權平均成績: {elective_and_general_average:.2f}")


<ipython-input-16-397cf5fa2d76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


1021(01) 學期 - 前5%學生平均成績：
必修科目加權平均成績: 90.36
選修和通識科目加權平均成績: 89.42


<ipython-input-16-397cf5fa2d76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


1022(01) 學期 - 前5%學生平均成績：
必修科目加權平均成績: 91.05
選修和通識科目加權平均成績: 90.34


<ipython-input-16-397cf5fa2d76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


...

1062(01) 學期 - 前5%學生平均成績：
必修科目加權平均成績: 91.89
選修和通識科目加權平均成績: 89.80


<ipython-input-16-397cf5fa2d76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


1071(01) 學期 - 前5%學生平均成績：
必修科目加權平均成績: 91.25
選修和通識科目加權平均成績: 90.67


<ipython-input-16-397cf5fa2d76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


1072(01) 學期 - 前5%學生平均成績：
必修科目加權平均成績: 92.37
選修和通識科目加權平均成績: 91.16


In [17]:
import pandas as pd
import numpy as np

# 定義加權平均函數
def weighted_average(df):
    if not df.empty:
        return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
    else:
        return np.nan

# 學期代碼列表
semesters = ["1021(01)"]

# 部門列表
departments = ['企業管理學系']

# 遍歷每個學期進行分析
for semester in semesters:
    # 讀取學生平均成績數據和選課資料
    url_1 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E5%AD%B8%E6%9C%9F%E5%B9%B3%E5%9D%87%E6%88%90%E7%B8%BE_{semester}.csv"
    df_1 = pd.read_csv(url_1)
    df_filtered_1 = df_1[df_1['系所名稱'].isin(departments)]
    df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')
    df_filtered_1 = df_filtered_1.dropna(subset=['學期-班一般人數', '學期-班一般名次'])

    url_2 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E9%81%B8%E8%AA%B2%E6%88%90%E7%B8%BE%E8%B3%87%E6%96%99_{semester}.csv"
    df_2 = pd.read_csv(url_2)

    # 篩選前5%的學生
    top_students = pd.DataFrame()
    for (dept, year_class), group in df_filtered_1.groupby(['系所名稱', '年級班別']):
        percentile_5 = max(int(round(len(group) * 0.05)), 1)
        top_students_in_group = group.nsmallest(percentile_5, '學期-班一般名次')
        top_students = pd.concat([top_students, top_students_in_group], ignore_index=True)

    top_student_ids = top_students['學號'].tolist()

    # 針對每位學生計算加權平均成績
    for student_id in top_student_ids:
        student_courses = df_2[df_2['學號'] == student_id]
        required_courses = student_courses[student_courses['課程屬性名稱'] == '必修']
        elective_courses = student_courses[student_courses['課程屬性名稱'] == '選修']
        general_courses = student_courses[student_courses['課程屬性名稱'] == '通識']

        required_average = weighted_average(required_courses)
        elective_and_general_average = weighted_average(pd.concat([elective_courses, general_courses]))

        print(f"學號: {student_id}")
        print(f"必修科目加權平均成績: {required_average:.2f}")
        print(f"選修和通識科目加權平均成績: {elective_and_general_average:.2f}")



<ipython-input-17-d0b1e5d0132b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')


學號: 9e3b9ef3f225ac16afb81d2e0ea34ee51644ed6774850b7d3b54c6cb3d639622
必修科目加權平均成績: 93.75
選修和通識科目加權平均成績: 92.00
學號: 6e86aa4d8503522c57ea2d3a3aa196d057a8d593d7fd31450c89e54dfe97c453
必修科目加權平均成績: 92.14
選修和通識科目加權平均成績: 82.00
學號: a286d34cdab17620b55cf38bb37002e3cba719d37c49ffb27df207a9125948cc
必修科目加權平均成績: 89.43
選修和通識科目加權平均成績: 89.00
學號: 02808c7ec1c82aac497c6f34180dfc8e28133a074e2e89532d3999b872da3412
必修科目加權平均成績: 95.57
選修和通識科目加權平均成績: 87.00
學號: 13d4d271779097e496d628f8aea7dc7346d58aa2ff77282aaac9a149f04de7a2
必修科目加權平均成績: 90.86
選修和通識科目加權平均成績: 88.60
學號: c00ba5d2b3e00fa51b2d0993d0896037b2e0e2b48995b070d047a3ec94a2436b
必修科目加權平均成績: 92.29
選修和通識科目加權平均成績: 82.60
學號: 243e0b67bd85ae3308482494f58295ae1f58ec775da2003b5fc01980aeff1f23
必修科目加權平均成績: 92.40
選修和通識科目加權平均成績: 93.50
學號: e150c3245190537b4ac25f62902770d3c198939f84a7b7101612eb96879e0c08
必修科目加權平均成績: 91.00
選修和通識科目加權平均成績: 92.00
學號: 61d27073c4f006fc35816ed1bb490ef17070647c6f238aeb813b7ea13a916477
必修科目加權平均成績: 91.14
選修和通識科目加權平均成績: nan
學號: cae5d2b9b933b9b6afe0432541

In [26]:
import pandas as pd
import numpy as np

# 定義加權平均函數
def weighted_average(df):
    if not df.empty:
        return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
    else:
        return np.nan

# 學期代碼列表
semesters = ["1021(01)", "1022(01)", "1031(01)", "1032(01)", "1041(01)", "1042(01)",
    "1051(01)", "1052(01)", "1061(01)", "1062(01)", "1071(01)", "1072(01)"]

# 部門列表
departments = ['資訊管理學系', '文學院學士班', '中國文學系', '英美語文學系', '法國語文學系',
    '理學院學士班', '物理學系', '數學系', '化學學系', '光電科學與工程學系',
    '工學院學士班', '化學工程與材料工程學系', '土木工程學系', '機械工程學系']

# 計數器
total_top_5_percent_students = 0
count_required_higher = 0

# 遍歷每個學期進行分析
for semester in semesters:
    for department in departments:
        # 讀取學生平均成績數據和選課資料
        url_1 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E5%AD%B8%E6%9C%9F%E5%B9%B3%E5%9D%87%E6%88%90%E7%B8%BE_{semester}.csv"
        df_1 = pd.read_csv(url_1)

        if '學號' not in df_1.columns:
            print(f"Column '學號' does not exist in the dataset for semester {semester} and department {department}.")
            continue

        df_filtered_1 = df_1.loc[df_1['系所名稱'] == department].copy()
        df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce').fillna(0)

        url_2 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E9%81%B8%E8%AA%B2%E6%88%90%E7%B8%BE%E8%B3%87%E6%96%99_{semester}.csv"
        df_2 = pd.read_csv(url_2)

        # 篩選前5%的學生
        top_students = pd.DataFrame()
        for (dept, year_class), group in df_filtered_1.groupby(['系所名稱', '年級班別']):
            percentile_5 = max(int(np.ceil(len(group) * 0.05)), 1)
            top_students_in_group = group.nsmallest(percentile_5, '學期-班一般名次')
            top_students = pd.concat([top_students, top_students_in_group], ignore_index=True)

        # 更新5%以內學生總人數
        total_top_5_percent_students += len(top_students)

        # 確保學號列存在
        if '學號' in top_students.columns:
            top_student_ids = top_students['學號'].tolist()

            # 針對每位學生計算加權平均成績
            for student_id in top_student_ids:
                student_courses = df_2[df_2['學號'] == student_id]
                required_courses = student_courses[student_courses['課程屬性名稱'] == '必修']
                elective_courses = student_courses[student_courses['課程屬性名稱'] == '選修']
                general_courses = student_courses[student_courses['課程屬性名稱'] == '通識']

                required_average = weighted_average(required_courses)
                elective_and_general_average = weighted_average(pd.concat([elective_courses, general_courses]))

                if required_average > elective_and_general_average:
                    count_required_higher += 1

# 最後輸出統計結果
print(f"所有學期和科系中，前5%學生的總人數: {total_top_5_percent_students}")
print(f"所有學期和科系中，必修課分數大於選修和通識課的學生人數: {count_required_higher}")



<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars

所有學期和科系中，前5%學生的總人數: 2342
所有學期和科系中，必修課分數大於選修和通識課的學生人數: 1050


<ipython-input-26-3a581b86371c>:7: RuntimeWarning: invalid value encountered in double_scalars
  return (df['成績'] * df['學分數']).sum() / df['學分數'].sum()
